In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import urllib.request
from scipy.stats import mode

from multiprocessing import Pool

In [3]:
rules_file = open("dane/ClassificationRules.txt","r")
rules_data = rules_file.readlines()
rules = np.empty((266,9350))
for i in range(266):
     rules[i, :] = list(map(float, rules_data[i].split()))
        

expert_file = open("dane/ImageExpertReduced.txt","r")
expert = expert_file.readlines()
expert = np.array([list(map(float, expert[0].split()))])

print(expert.shape)
print(rules.shape)

(1, 9350)
(266, 9350)


In [4]:
def random_individual(p,d):
    random_vec = np.random.rand(d)
    return (random_vec < p).astype(int)

def random_population(p,d,N):
    return np.array([random_individual(p,d) for i in range(N)])

In [5]:
most_voted = lambda v: mode(v)[0]

def opt_f(x):
    x = x[np.newaxis,:]
    rules_in_use = rules[np.nonzero(x)[1]]
    votes = np.array([most_voted(v) for v in rules_in_use.T])
    votes = votes.T
    return ((votes == expert).sum() / expert.shape[1])

In [6]:
def PBIL(d,N,t1,t2,t3,T,threads):

    costs = np.zeros((3,T))
    bests = np.zeros((T,d))
    p_arr = np.zeros((T,d))
    
    p = np.ones(d)
    p = p/2
    P = random_population(p,d,N)
    last = time.time()
    for t in range(T):
        
        val = np.empty(1)
        with Pool(threads) as pool:
            val = np.array(pool.map(opt_f, P))
            pool.close()
            pool.join()
        x_best = P[val.argmax()]

        costs[0][t] = val.max()
        costs[1][t] = val.mean()
        costs[2][t] = val.min()

        bests[t] = x_best
        p_arr[t] = p
        
        for i in range(N):
            p = p * (1 - t1) + x_best * t1

        for j in range(N):
            if np.random.rand() < t2:
                p = p * (1 - t3) + random_individual((np.ones(d)/2),d) * t3

        P = random_population(p,d,N)
        
        print(t, time.time() - last)
        last = time.time()
    return (costs,bests,p_arr)

In [7]:
#data = PBIL(266,50,0.045,0.015,0.02,900)u